In [4]:
# please run this exp in sagemath
from math import sqrt,gcd
from Crypto.Util.number import *
from sympy import nextprime
from sympy import primerange

def PrimeN(n):
    a = [0]*(n+1)
    a[0] = a[1] = 0
    for i  in range(2,n+1):
        a[i] = 1
    for i in range(2,int(sqrt(n)+1)):
        mul = 2
        if (a[i] == 0):
            continue
        while (i * mul <= n):
            a[i * mul] = 0
            mul+=1
    return a

def max_order_element_combine(n,nums=1):
    #MX = [1]*(n+1)
    prime = PrimeN(n)
    prime_pows = {}
    for i in range(n,-1,-1):
        if not prime[i]:continue
        k = i*i
        prime_pows.setdefault(i,i)
        while (k <= n):
            prime[i]+=1
            prime_pows.setdefault(k,k)
            k *= i
    res = []
    for _ in range(nums):
        MX = [1]*(n+1)
        for i in range(2,n+1):
            if not prime[i]:continue
            for j in range(n,1,-1):
                temp = i
                for k in range(1,prime[i]+1):
                    if (j - temp >= 0 and MX[j] < MX[j - temp] * prime_pows[temp]):
                        MX[j] = MX[j - temp] * prime_pows[temp]
                    temp *= i
                    
        res.append(MX[-1])
        res_facts = factor(res[-1])
        # renew the bag weights
        for f in res_facts:
            p_f =f[0]**f[1]
            j = f[0]
            while j < n:
                if j <= p_f:
                    prime_pows[j] = 1
                else:
                    prime_pows[j] = min(j//p_f,prime_pows[j])
                j*=f[0]
    return res

def aupton_2(N): # compute terms a(0)..a(N)
    V = [1 for j in range(2*N+1)]
    for i in primerange(2, N+1):
        for j in range(2*N, i-1, -1):
            hi = V[j]
            pp = i
            while pp <= j:
                hi = max((pp if j==pp else V[j-pp]*pp), hi)
                pp *= i
            V[j] = hi
    return V
'''
Wikipedia subset sum approximation algorithm
http://en.wikipedia.org/wiki/Subset_sum_problem#Polynomial_time_approximate_algorithm
'''

import operator
def approx_with_accounting_and_duplicates(x_list,s):
    c = .01              # fraction error (constant)
    N = len(x_list)      # number of values

    S = [(0, [])]
    for x in sorted(x_list):
        T = []
        for y, y_list in S:
            T.append((x + y, y_list + [x]))
        U = T + S
        U = sorted(U, key=operator.itemgetter(0))
        y, y_list = U[0]
        S = [(y, y_list)]

        for z, z_list in U:
            lower_bound = (float(y) + c * float(s) / float(N))
            if lower_bound < z <= s:
                y = z
                S.append((z, z_list))

    return sorted(S, key=operator.itemgetter(0))[-1]

def split_2n(n,order):
    num_set = set([p^e for p,e in factor(order)])
    target_sum = n
    sum1,prime_list1 = approx_with_accounting_and_duplicates(num_set,target_sum)
    prime_list2 = num_set - set(prime_list1)
    sum2 = sum(prime_list2)
    if sum1 <=  target_sum and sum2 <= target_sum:
        return prod(prime_list1),prod(prime_list2)
    
def Landu_expand(n):
    order = aupton_2(n)[-1]
    return split_2n(n,order)

In [5]:
# permutation discrete

from pwn import *
from Crypto.Util.number import *
from Crypto.Cipher import AES
import os
from math import factorial
import pickle
from random import sample
from random import randint
import re

s2n=lambda x: [int(x) for x in re.findall(r"\-?\d+\.?\d*",x)]

# 202.38.93.111 10114
io =  remote("202.38.93.111",10114)
token = "819:MEQCIF5Ohbnr1w9u3xhZE6fM8qZjDjbi7BgIa0ammpOdASt6AiBaFF4YoiOD1eYhB+dP8wrcqYJTF8MIplFGIO3Y29Xffw=="
io.sendlineafter(b"Please input your token: ",token.encode())

def solve1():
    io.sendlineafter(b"> your choice: ", b"1")
    for _ in range(15):
        io.recvuntil(b"[+] RSA public key: ")
        n,e = s2n(io.recvline().decode().strip())
        G = SymmetricGroup(n)
        io.recvline()
        enc = G(s2n(io.recvline().decode().strip()))
        phi = enc.order()
        s = enc^(inverse(e,phi))
        io.sendlineafter(b"> your answer: ",str(s.tuple()).encode())
        print(io.recvline())
    print(io.recvline())
    
# solve1()   

def solve2():
    io.sendlineafter(b"> your choice: ", b"2")
    for _ in range(15):
        io.recvuntil(b"DH public key: ")
        PubKey = s2n(io.recvline().decode().strip())
        n,g = PubKey[0],PubKey[1:]
        G = SymmetricGroup(n)
        g = G(g)
        Y = G(s2n(io.recvline().decode().strip()))
        secret = discrete_log(Y,g) 
        print(f"[+] crack secrete: {secret}")
        io.sendlineafter(b"> your answer: ",str(secret).encode())
        print(io.recvline())
    print(io.recvline())
    
def random_perm(n,order):
    factors = factor(order)
    tuple_len = [p^e for p,e in factors]
    remained = list(range(1,n+1))
    result = [0]*n
    for tl in tuple_len:
        sub = sample(remained,tl)
        for i in range(tl):
            result[sub[i]-1]=sub[(i+1)%tl]
        remained = [x for x in remained if x not in sub]
        
    while result.count(0)!=0:
        sub = sample(list(remained),1)
        result[sub[0]-1]=sub[0]
        remained = [x for x in remained if x not in sub]
    L = sorted(result)
    for i in range(1,n+1):
        assert int(i) == int(L[i-1]),f"{i}"
    return result
        
    
def solve3():
    io.sendlineafter(b"> your choice: ", b"3")
    for _ in range(15):
        io.recvuntil(b"DH public key: ")
        n = s2n(io.recvline().decode().strip())[0]
        G = SymmetricGroup(n)
        secret_bound = s2n(io.recvline().decode().strip())[0]
        # original solver 
        # order1,order2 = max_order_element_combine(n,2)
        # new bound
        order1,order2 = Landu_expand(n)
        print(f"[+] crack with probability {int(lcm([order1,order2]))/int(secret_bound)}")
        g1 = random_perm(n,order1)
        g2 = random_perm(n,order2)
        io.sendlineafter(b"(a list): ",str(g1).encode())  
        Y1 = s2n(io.recvline().decode().strip())[1:]
        io.sendlineafter(b"(a list): ",str(g2).encode())
        Y2 = s2n(io.recvline().decode().strip())[1:]
        Y1,g1,Y2,g2 = G(Y1),G(g1),G(Y2),G(g2)
        dlp1 = discrete_log(Y1,g1)
        dlp2 = discrete_log(Y2,g2)
        recovered_secrert = crt([dlp1,dlp2],[order1,order2])
        io.sendlineafter(b"> your answer: ",str(recovered_secrert).encode())
        print(io.recvline())
    print(io.recvline())

solve1()
solve2()
solve3()

[x] Opening connection to 202.38.93.111 on port 10114
[x] Opening connection to 202.38.93.111 on port 10114: Trying 202.38.93.111
[+] Opening connection to 202.38.93.111 on port 10114: Done
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'Good job\n'
b'flag{Pe5mRSA?__1s_s00oo_weak!!!_35c4d0c866}\n'
[+] crack secrete: 3154375377
b'Good job\n'
[+] crack secrete: 3079413974
b'Good job\n'
[+] crack secrete: 2651149421292
b'Good job\n'
[+] crack secrete: 17702597092
b'Good job\n'
[+] crack secrete: 77057294432
b'Good job\n'
[+] crack secrete: 15111282991
b'Good job\n'
[+] crack secrete: 3757428107607
b'Good job\n'
[+] crack secrete: 41130018764
b'Good job\n'
[+] crack secrete: 11020006428
b'Good job\n'
[+] crack secrete: 68726335957
b'Good job\n'
[+] crack secrete: 29045566010013
b'Good job\n'
[+] crack secrete: 8008357749
b'Good job\n'
[+] cra